In [1]:
import sys
import io
import numpy as np
from requests.auth import HTTPBasicAuth

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import datetime
from datetime import timedelta
import math

from folium import plugins
from folium.plugins import HeatMapWithTime
import folium 

import webbrowser

sys.path.insert(1, '../')
from get_data_API import get_data

In [2]:
from jinja2 import Template 
from folium.map import Layer

class HeatMapWithTimeAdditional(Layer):
    _template = Template("""
        {% macro script(this, kwargs) %}
            var {{this.get_name()}} = new TDHeatmap({{ this.data }},
                {heatmapOptions: {
                    radius: {{this.radius}},
                    minOpacity: {{this.min_opacity}},
                    maxOpacity: {{this.max_opacity}},
                    scaleRadius: {{this.scale_radius}},
                    useLocalExtrema: {{this.use_local_extrema}},
                    defaultWeight: 1,
                    {% if this.gradient %}gradient: {{ this.gradient }}{% endif %}
                }
            }).addTo({{ this._parent.get_name() }});
        {% endmacro %}
    """)

    def __init__(self, data, name=None, radius=15,
                 min_opacity=0, max_opacity=0.6,
                 scale_radius=False, gradient=None, use_local_extrema=False,
                 overlay=True, control=True, show=True):
        super(HeatMapWithTimeAdditional, self).__init__(
            name=name, overlay=overlay, control=control, show=show
        )
        self._name = 'HeatMap'
        self.data = data

        # Heatmap settings.
        self.radius = radius
        self.min_opacity = min_opacity
        self.max_opacity = max_opacity
        self.scale_radius = 'true' if scale_radius else 'false'
        self.use_local_extrema = 'true' if use_local_extrema else 'false'
        self.gradient = gradient

In [3]:
# get data
df = get_data()

200


In [4]:
df

,ID,time,temperature,humidity,light,motion,co2,coordinates
0,a8-17-58-ff-fe-03-0f-f9,2023-04-03T00:00:08.007Z,21.4,14,3,0,299,"[65.05994, 25.46653]"
1,a8-17-58-ff-fe-03-0f-ac,2023-04-03T00:00:09.103Z,21.0,13,1,0,559,"[65.05785, 25.46718]"
2,a8-17-58-ff-fe-03-10-8c,2023-04-03T00:00:13.220Z,20.5,13,1,2,521,"[65.05806, 25.46922]"
3,a8-17-58-ff-fe-03-10-91,2023-04-03T00:00:15.757Z,21.3,10,11,0,454,"[65.05849, 25.46598]"
4,a8-17-58-ff-fe-03-0f-8d,2023-04-03T00:00:18.178Z,20.1,12,5,0,567,"[65.05865, 25.4656]"
...,...,...,...,...,...,...,...,...
12699,a8-17-58-ff-fe-03-10-13,2023-04-03T11:22:17.415Z,20.2,16,3,0,464,"[65.05855, 25.46634]"
12700,a8-17-58-ff-fe-04-65-2d,2023-04-03T11:22:20.471Z,22.6,14,433,60,None,"[65.0583, 25.46677]"
12701,a8-17-58-ff-fe-04-1f-ee,2023-04-03T11:22:30.403Z,17.9,93,None,0,None,"[65.06343, 25.46504]"
12702,a8-17-58-ff-fe-03-0d-50,2023-04-03T11:22:31.715Z,20.9,12,54,26,367,"[65.05961, 25.4665]"


# Testing function code row by row

In [5]:
selected_data = df[["coordinates", "motion", "time"]]
selected_data

,coordinates,motion,time
0,"[65.05994, 25.46653]",0,2023-04-03T00:00:08.007Z
1,"[65.05785, 25.46718]",0,2023-04-03T00:00:09.103Z
2,"[65.05806, 25.46922]",2,2023-04-03T00:00:13.220Z
3,"[65.05849, 25.46598]",0,2023-04-03T00:00:15.757Z
4,"[65.05865, 25.4656]",0,2023-04-03T00:00:18.178Z
...,...,...,...
12699,"[65.05855, 25.46634]",0,2023-04-03T11:22:17.415Z
12700,"[65.0583, 25.46677]",60,2023-04-03T11:22:20.471Z
12701,"[65.06343, 25.46504]",0,2023-04-03T11:22:30.403Z
12702,"[65.05961, 25.4665]",26,2023-04-03T11:22:31.715Z


In [6]:
#Convert ISO-8601 time format to pandas DateTime format 

selected_data.time = pd.to_datetime(selected_data.time, format="%Y-%m-%dT%H:%M:%S.%fZ")
selected_data

C:\Users\Ville\AppData\Local\Temp\ipykernel_14524\3334907349.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_data.time = pd.to_datetime(selected_data.time, format="%Y-%m-%dT%H:%M:%S.%fZ")


,coordinates,motion,time
0,"[65.05994, 25.46653]",0,2023-04-03 00:00:08.007
1,"[65.05785, 25.46718]",0,2023-04-03 00:00:09.103
2,"[65.05806, 25.46922]",2,2023-04-03 00:00:13.220
3,"[65.05849, 25.46598]",0,2023-04-03 00:00:15.757
4,"[65.05865, 25.4656]",0,2023-04-03 00:00:18.178
...,...,...,...
12699,"[65.05855, 25.46634]",0,2023-04-03 11:22:17.415
12700,"[65.0583, 25.46677]",60,2023-04-03 11:22:20.471
12701,"[65.06343, 25.46504]",0,2023-04-03 11:22:30.403
12702,"[65.05961, 25.4665]",26,2023-04-03 11:22:31.715


In [7]:
new = selected_data.groupby(pd.Grouper(key='time', freq='15Min'),as_index=False).apply(lambda x: x['time'])
selected_data['period'] = new.index.get_level_values(0)
selected_data


C:\Users\Ville\AppData\Local\Temp\ipykernel_14524\3887612080.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_data['period'] = new.index.get_level_values(0)


,coordinates,motion,time,period
0,"[65.05994, 25.46653]",0,2023-04-03 00:00:08.007,0
1,"[65.05785, 25.46718]",0,2023-04-03 00:00:09.103,0
2,"[65.05806, 25.46922]",2,2023-04-03 00:00:13.220,0
3,"[65.05849, 25.46598]",0,2023-04-03 00:00:15.757,0
4,"[65.05865, 25.4656]",0,2023-04-03 00:00:18.178,0
...,...,...,...,...
12699,"[65.05855, 25.46634]",0,2023-04-03 11:22:17.415,45
12700,"[65.0583, 25.46677]",60,2023-04-03 11:22:20.471,45
12701,"[65.06343, 25.46504]",0,2023-04-03 11:22:30.403,45
12702,"[65.05961, 25.4665]",26,2023-04-03 11:22:31.715,45


In [8]:
selected_data['period'].iloc[-1]

45

In [9]:
#create time indexes for HeatMapWithTime
timeIndexes = []
selected_data['period'].iloc[-1]
#get last period in dataframe
for i in range(selected_data['period'].iloc[-1]):
    
    #Add multiples of 15min to first timestamp based on period value
    timeIndexes.append(selected_data["time"].iloc[1] + i * timedelta(minutes=15))
    
len(timeIndexes)

45

In [10]:
#Check amount of data points in each 15 minute segment
df2 = selected_data.pivot_table(index = ['period'], aggfunc ='size')
df2

period
0     277
1     294
2     275
3     279
4     289
5     294
6     287
7     288
8     277
9     286
10    288
11    287
12    280
13    289
14    286
15    286
16    286
17    275
18    280
19    278
20    277
21    276
22    279
23    276
24    281
25    275
26    275
27    269
28    275
29    281
30    285
31    281
32    270
33    273
34    273
35    278
36    273
37    270
38    272
39    278
40    282
41    279
42    272
43    267
44    279
45    127
dtype: int64

# Create function for python file

In [11]:
def data_formating(df, data_shown):
    
    print(f"Formating data for {data_shown} map")
    selected_data = df[["coordinates", data_shown, "time"]]
    
    #Convert ISO-8601 time format to pandas DateTime format 
    selected_data.time = pd.to_datetime(selected_data.time, format="%Y-%m-%dT%H:%M:%S.%fZ")
    
    #Add new column 'period' to dataframe, every row gets group number based on 15min clusters
    new = selected_data.groupby(pd.Grouper(key='time', freq='15Min'),as_index=False).apply(lambda x: x['time'])
    selected_data['period'] = new.index.get_level_values(0)
    
    #create time indexes for HeatMapWithTime ----------------------
    timeIndexes = []
    
    #get last period in dataframe
    for i in range(selected_data['period'].iloc[-1] + 1):

        #Add multiples of 15min to first timestamp based on period value
        timeIndexes.append((selected_data["time"].iloc[1] + i * timedelta(minutes=15)).strftime("%d/%m/%Y, %H:%M:%S")[:-3])

    #iterate through dataframe -------------------------------------
    data = [[] for _ in range(len(timeIndexes))]
    
    for _, row in selected_data.iterrows():
        #Checking for None type
        if not row[data_shown] is None:
            #Check that data isn't zero or NaN
            if (not math.isnan(row[data_shown]) and row[data_shown] != 0.0):
                
                data[row["period"]].append([row['coordinates'][0],row['coordinates'][1],row[data_shown]])
              
    return data, timeIndexes       
 

In [12]:
#Min-Max scale motion data
column = "motion"
df[column] = (df[column] - df[column].min()) / (df[column].max() - df[column].min())

#Create heatmap using motion data
motion_data, motion_time = data_formating(df, "motion")


#Create heatmap using co2 data
co2_data, co2_time = data_formating(df, "co2")

#Create heatmap using temperature data
temperature_data, temperature_time = data_formating(df, "temperature")

#Create heatmap using humidity data
humidity_data, humidity_time = data_formating(df, "humidity")


C:\Users\Ville\AppData\Local\Temp\ipykernel_14524\1537243491.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_data.time = pd.to_datetime(selected_data.time, format="%Y-%m-%dT%H:%M:%S.%fZ")
C:\Users\Ville\AppData\Local\Temp\ipykernel_14524\1537243491.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_data['period'] = new.index.get_level_values(0)


Formating data for motion map
Formating data for co2 map


C:\Users\Ville\AppData\Local\Temp\ipykernel_14524\1537243491.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_data.time = pd.to_datetime(selected_data.time, format="%Y-%m-%dT%H:%M:%S.%fZ")
C:\Users\Ville\AppData\Local\Temp\ipykernel_14524\1537243491.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_data['period'] = new.index.get_level_values(0)


Formating data for temperature map


C:\Users\Ville\AppData\Local\Temp\ipykernel_14524\1537243491.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_data.time = pd.to_datetime(selected_data.time, format="%Y-%m-%dT%H:%M:%S.%fZ")
C:\Users\Ville\AppData\Local\Temp\ipykernel_14524\1537243491.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_data['period'] = new.index.get_level_values(0)


Formating data for humidity map


C:\Users\Ville\AppData\Local\Temp\ipykernel_14524\1537243491.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_data.time = pd.to_datetime(selected_data.time, format="%Y-%m-%dT%H:%M:%S.%fZ")
C:\Users\Ville\AppData\Local\Temp\ipykernel_14524\1537243491.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_data['period'] = new.index.get_level_values(0)


In [13]:
m = folium.Map([65.059228, 25.465375], zoom_start=16.45)

#Motion
HeatMapWithTime(motion_data, index=motion_time, auto_play=True, max_opacity=0.5, name="Motion").add_to(m)
#CO2
HeatMapWithTimeAdditional(co2_data, show=False, max_opacity=0.5, name="CO2").add_to(m)
#Temperature
HeatMapWithTimeAdditional(temperature_data, show=False, max_opacity=0.5, name="temperature").add_to(m)
#Humidity
HeatMapWithTimeAdditional(humidity_data, show=False, max_opacity=0.5, name="humidity").add_to(m)


folium.LayerControl().add_to(m)

m

In [14]:
m.save(outfile="index.html")

In [15]:
#Display html map
webbrowser.open("index.html")

True